<a href="https://colab.research.google.com/github/XueqingWu/Explainable_AI_in_LLM/blob/main/XAI_in_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing Embedding (Latent) Space

In [1]:
pip install umap

  Preparing metadata (setup.py) ... done
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3542 sha256=18b52cd65790fba6d200aa23ee561056255eb2c7945fc2af4ece463e8fa5a37e
  Stored in directory: /root/.cache/pip/wheels/15/f1/28/53dcf7a309118ed35d810a5f9cb995217800f3f269ab5771cb
Successfully built umap


In [2]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 10.5 MB/s eta 0:00:00


In [3]:

# Basic
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

# Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

# Embeddings
import gensim.downloader as api


## Import LLM
In this project, we are using **stella_en_400M_v5** model from MTEB leaderboard on Huggingface. The model directory structure is very simple, it is a standard SentenceTransformer directory with a series of 2_Dense_{dims} folders, where dims represents the final vector dimension. We are using 1024 dimension in thie project.

The model simplifies usage of prompts, providing two prompts for most general tasks, one is for s2p, another one is for s2s.

Need to use **GPU** to run the code of this project.

In [4]:
pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 54.0 MB/s eta 0:00:00


In [5]:
from sentence_transformers import SentenceTransformer

# This model supports two prompts: "s2p_query" and "s2s_query" for sentence-to-passage and sentence-to-sentence tasks, respectively.
# They are defined in `config_sentence_transformers.json`
query_prompt_name = "s2p_query"
queries = [
    "What are some ways to reduce stress?",
    "What are the benefits of drinking green tea?",
]
# docs do not need any prompts
docs = [
    "There are many effective ways to reduce stress. Some common techniques include deep breathing, meditation, and physical activity. Engaging in hobbies, spending time in nature, and connecting with loved ones can also help alleviate stress. Additionally, setting boundaries, practicing self-care, and learning to say no can prevent stress from building up.",
    "Green tea has been consumed for centuries and is known for its potential health benefits. It contains antioxidants that may help protect the body against damage caused by free radicals. Regular consumption of green tea has been associated with improved heart health, enhanced cognitive function, and a reduced risk of certain types of cancer. The polyphenols in green tea may also have anti-inflammatory and weight loss properties.",
]

# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# on gpu
model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).cuda()
# you can also use this model without the features of `use_memory_efficient_attention` and `unpad_inputs`. It can be worked in CPU.
# model = SentenceTransformer(
#     "dunzhang/stella_en_400M_v5",
#     trust_remote_code=True,
#     device="cpu",
#     config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False}
# )
query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
doc_embeddings = model.encode(docs)
print(query_embeddings.shape, doc_embeddings.shape)
# (2, 1024) (2, 1024)

similarities = model.similarity(query_embeddings, doc_embeddings)
print(similarities)
# tensor([[0.8398, 0.2990],
#         [0.3282, 0.8095]])


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/169k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/186 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

(2, 1024) (2, 1024)
tensor([[0.8398, 0.2990],
        [0.3282, 0.8095]])


In [15]:
# Select a subset of words for visualization
subset_size = 500

# Define a subset of words to encode as separate "queries" for embeddings
words = ["reduce", "stress", "green", "tea", "benefits", "health"]
# subset_size = min(500, len(words))  # Ensure subset size doesn't exceed available words
words_subset = words[:subset_size]

# Get embeddings for each word (processed as single-element "queries")
embeddings = np.array([model.encode([word])[0] for word in words_subset])

# Now you can use `word_embeddings` for any further analysis or similarity computations
print(embeddings.shape)  # Shape should be (subset_size, 1024)

(6, 1024)


I have constructed a list of words that consists of 6 words, which are related to the mental health and tea (extracted the theme from queries and docs above). I want to see the similarites of the embeddings between those words.

## Principal Component Analysis (PCA)

*   Focus on capturing global linear relationships in the data
*   Use to: simplify and find global linear relationships and patterns in the data

**How does PCA work?**


1. **Standardize the Data:** Scale the data so each feature has a mean of 0 and standard deviation of 1
2. **Compute the Covariance Matrix:** Calculate the covariance matrix to understand how features vary together
3. **Compute Eigenvalues and Eigenvectors:** Derive the eigenvalues and eigenvectors from the covariance matrix. Eigenvectors represent principal components, and eigenvalues indicate their significance
4. **Sort Eigenvalues and Eigenvectors:** Order them by descending eigenvalues to prioritize the most significant components
5. **Select Principal Components:** Choose the top 𝑘 eigenvectors corresponding to the largest eigenvalues
6. **Transform the Data:** Project the original data onto the selected principal components to reduce dimensions

In [16]:

# Apply PCA
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings)

# Plot PCA results using Plotly for interactivity
fig_pca = px.scatter(
    embeddings_pca, x=0, y=1,
    text=words,
    title="PCA of Stella_en Embeddings",
    labels={'0': 'Principal Component 1', '1': 'Principal Component 2'}
)
fig_pca.update_traces(marker=dict(size=8))
fig_pca.show()

According to PCA, the words "green" and "tea" are very similar for both pricipal components. "health", "benefits", "stress", and "reduce" are quite similar for principal component 1. However, "reduce" is far away from other three words according to principal component 2.

To imply form the image, the words in cluster of "health", "benefits", "stress" are kind of related with positive word meaning, where "reduce" are relatively negative. In contrast to those words that have sentiments, words like "tea" and "green" are more neutral. To sum up, the dimension reduction for PCA is leaning toward the global meaning.

### t-distributed Stochastic Neighbor Embedding (t-SNE)



* Constructs a lower-dimensional representation where similar data points are placed closer together
* Use to: Emphasize visualization, reveal local patterns and clusters



**How does t-SNE work?**

1. **Compute Pairwise Similarities:** Measure how similar each pair of data points is in the high-dimensional space using a Gaussian kernel
2. **Initialize Embeddings:** Start with random low-dimensional embeddings for each data point
3. **Compute Similarities in Low-Dimensional Space:** Measure similarities between low-dimensional embeddings using a Student's t-distribution
4. **Optimize Embeddings:** Adjust the embeddings to minimize the difference between the distributions of similarities in high-dimensional and low-dimensional spaces
5. **Reduce Dimensionality:** Obtain a reduced-dimensional representation of the data, preserving local relationships between data points

In [17]:
# Apply t-SNE
tsne = TSNE(n_components=2, perplexity=5, n_iter=300, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

# Plot t-SNE results using Plotly
fig_tsne = px.scatter(
    embeddings_tsne, x=0, y=1,
    text=words,
    title="t-SNE of Stell_en Embeddings",
    labels={'0': 'Component 1', '1': 'Component 2'}
)
fig_tsne.update_traces(marker=dict(size=8))
fig_tsne.show()

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



For t-SNE, all 6 words are quite scattered for both component 1 and component 2, implying low similairites among those words. t-SNE focuses more on the local structure.

### Umap
* Uses manifold learning (nonlinear dimensionality reduction) to understand the underlying structure or shape of the data
* Focus on capturing complex, non-linear relationships in the data
Use to: preserve local structure and handle complex, nonlinear relationships

**How does UMAP work?**


1. **Construct Local Neighborhoods:** Define local neighborhoods for each data point in the high-dimensional space based on proximity
2. **Optimize Low-Dimensional Embeddings:** Minimize the discrepancy between local neighborhoods in the high-dimensional and low-dimensional spaces using stochastic gradient descent
3. **Preserve Global Structure:** Balance the preservation of local and global structures using a fuzzy simplicial set representation
4. **Reduce Dimensionality:** Obtain a lower-dimensional representation of the data while preserving both local and global relationships
5. **Effective Visualization:** UMAP provides an effective tool for visualizing high-dimensional data in a reduced-dimensional space, capturing complex relationships and structures








In [18]:
import umap

In [19]:

# # Apply UMAP
# umap_model = umap.UMAP(n_components=2, n_neighbors=4, min_dist=0.1, random_state=42)
# embeddings_umap = umap_model.fit_transform(embeddings)

# # Plot UMAP results using Plotly
# fig_umap = px.scatter(
#     embeddings_umap, x=0, y=1,
#     text=words,
#     title="UMAP of GloVe Embeddings",
#     labels={'0': 'Component 1', '1': 'Component 2'}
# )
# fig_umap.update_traces(marker=dict(size=8))
# fig_umap.show()

In [20]:
pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.7 MB/s eta 0:00:00


In [22]:
# Import UMAP correctly
import umap.umap_ as umap
import plotly.express as px  # Import Plotly Express

# Apply UMAP
umap_model = umap.UMAP(n_components=2, n_neighbors=4, min_dist=0.1, random_state=42)
embeddings_umap = umap_model.fit_transform(embeddings)

# Plot UMAP results using Plotly
fig_umap = px.scatter(
    x=embeddings_umap[:, 0], y=embeddings_umap[:, 1],  # Separate the components
    text=words,
    title="UMAP of Stella_en Embeddings",
    labels={'x': 'Component 1', 'y': 'Component 2'}
)
fig_umap.update_traces(marker=dict(size=8))
fig_umap.show()


/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



According to UMAP, the words are quite scattered according to both components. UMAP focuses more on the local structure.

## Results Summary

The words show some clusters for PCA method, whereas those words are quite scattered for t-SNE and UMAP. However, the way t-SNE and UMAP scattered are quite different. The relative position of each word in t-SNE is largely different from UMAP.

To sum up, the PCA focuses more on finding global trends of words, whereas t-SNE focus more on the local structure.

# References
1. **Code Structure** (Demo Notebook): https://github.com/AIPI-590-XAI/Duke-AI-XAI/blob/main/explainable-ml-example-notebooks/embedding-visualization.ipynb
2. **Embedding Model:** https://huggingface.co/dunzhang/stella_en_400M_v5
3. **Model Importing and Code Revise:** ChatGPT